In [ ]:
pip install rdflib

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
from rdflib import Graph, Literal, RDF, URIRef, Namespace 
from rdflib.namespace import FOAF , XSD, DC, FOAF, SKOS, RDF, RDFS
import urllib.parse 
from google.colab import files 
import io
import re

In [ ]:
uploaded = files.upload()

In [ ]:
orgs = pd.read_excel('SP_DownloadLijst_LF_Organisaties_WIP_20210108.xlsx', sheet_name='DATA')

In [ ]:
gp = pd.read_excel('gemeente-provincie.xlsx', sheet_name='Feuil2')

# Data cleansing

## KOB help function

In [ ]:
def kbo_cleansing(kbo):
  kbo_cleansed = comment = np.nan

  if kbo != 'nan':
    kbo = re.sub(r'\D', '', kbo)
    if  re.match(r'\d{10}', kbo):
      kbo_cleansed = kbo
    elif re.match(r'\d{1,9}', kbo):
      kbo_cleansed = kbo
      comment = f'only {len(kbo)} digits.'
    else: 
      comment = 'Wrong KBO format. Check it.'
  else :
    comment = 'No KBO nr found'

  return [kbo_cleansed, comment]

## KBO numbers

In [ ]:
orgs[['KBOnr_cleansed', 'KBOnr_comment']] = pd.DataFrame(orgs['KBOnr'].astype(str).apply(kbo_cleansing).values.tolist(), columns=['kbo_cleansed','comment'])

In [ ]:
orgs['organisation_id'] = orgs['KBOnr_cleansed'].fillna(orgs['Unieke Naam'])

151 organisation whithout KBOnr and Unieke Naam

In [ ]:
orgs[(orgs['KBOnr_cleansed'].isnull()) & (orgs['Unieke Naam'].isnull())]

In [ ]:
len(orgs['organisation_id'].unique())

ORGS with wrong kbo entry (not 10 digits): **14 records**


In [ ]:
orgs[~(orgs['KBOnr_cleansed'].str.len() == 10) & (orgs['KBOnr_cleansed'].notnull())][['KBOnr_cleansed', 'KBOnr_comment']]

In [ ]:
orgs[orgs['KBOnr_comment'] == 'Wrong KBO format. Check it.']

## Website

In [ ]:
def website_cleansing(website):
  website_cleansed = comment = np.nan

  if website != 'nan':
    if  re.match(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,4}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)', website):
      website_cleansed = website
    else: 
      comment = 'Wrong website format. Check it.'

  return [website_cleansed, comment]



In [ ]:
orgs[['Website Cleansed', 'Website Comment']] = pd.DataFrame(orgs['Website'].astype(str).apply(website_cleansing).values.tolist(), columns=['website_cleansed','comment'])

## Unieke Naam van actieve organisaties

In [ ]:
orgs['Unieke Naam van actieve organisaties'] = orgs['Unieke Naam van actieve organisaties'].astype(str).str.replace('0', '').replace('nan', None)

## Province

In [ ]:
orgs['Provincie van de organisatie']

In [ ]:
orgs['Provincie van de organisatie'].dropna().unique()

In [ ]:
len(orgs['Gemeente van de organisatie'].dropna().unique())

In [ ]:
orgs[orgs['Gemeente van de organisatie'] == 'Brussel']

In [ ]:
orgs[orgs['Gemeente van de organisatie'] == 'Brussel']['Provincie van de organisatie']

In [ ]:
def find_city_provincie(city):
  return gp[gp['Gemeente'].str.contains(city)]

In [ ]:
orgs['Provincie van de organisatie_cleansed'] = None
orgs['Provincie van de organisatie_comment'] = None

In [ ]:
for index, row in orgs.iterrows():
  city = str(row['Gemeente van de organisatie']).strip().title()
  result = find_city_provincie(city)
  
  if len(result) > 0:
    if str(result.iloc[0]['Provincie']) != str(row['Provincie van de organisatie']):
      orgs.at[index, 'Provincie van de organisatie_cleansed'] = result.iloc[0]['Provincie']
      orgs.at[index, 'Provincie van de organisatie_comment'] = "Different Provincie"
    else:
      orgs.at[index, 'Provincie van de organisatie_cleansed'] = row['Provincie van de organisatie']
  elif city != 'NaN':
    orgs.at[index, 'Provincie van de organisatie_comment'] = "Municipality Not Found"
    orgs.at[index, 'Provincie van de organisatie_cleansed'] = row['Provincie van de organisatie']

In [ ]:
orgs.info()

In [ ]:
#orgs['Provincie van de organisatie_cleansed'] = np.where(orgs['Gemeente van de organisatie'] == 'Brussel', 'VLAAMS-BRABANT', orgs['Provincie van de organisatie'])
#orgs['Provincie van de organisatie_comment'] = np.where(orgs['Provincie van de organisatie'] != 'VLAAMS-BRABAN', 'Wrong Provincie', '')

In [ ]:
orgs[['Gemeente van de organisatie', 'Provincie van de organisatie', 'Provincie van de organisatie_cleansed', 'Provincie van de organisatie_comment']]

Wrong Provincie

In [ ]:
orgs[orgs['Provincie van de organisatie_comment'] == 'Different Provincie'][['Gemeente van de organisatie', 'Provincie van de organisatie', 'Provincie van de organisatie_cleansed', 'Provincie van de organisatie_comment']]

In [ ]:
df = orgs[orgs['Provincie van de organisatie_comment'] == 'Different Provincie']

In [ ]:
df.to_excel("different_provincies.xlsx")  

City not found

In [ ]:
orgs[orgs['Provincie van de organisatie_comment'] == 'Municipality Not Found'][['Gemeente van de organisatie', 'Provincie van de organisatie', 'Provincie van de organisatie_cleansed', 'Provincie van de organisatie_comment']]

In [ ]:
df_municipalities = orgs[orgs['Provincie van de organisatie_comment'] == 'Municipality Not Found']

In [ ]:
df_municipalities.to_excel("municipality_not_found.xlsx")

Cities not found in the list **12**

Borgerhout: Neighborhood in Antwerp. *Change it to Antwerp?*

Hoboken: District in Antwerp. *Should it be  Antwerp (Hoboken)?*

Kallo: District in Beveren. *Should it be  Beveren (Kallo)*

Zellik: District in Asse. *Should it be Asse (Zellik)*

Luik: Dutch version name for Liège?

Westmeerbeek: District in Hulshout? *Should it be Hulshout (Westmeerbeek)?*

Sint-Andries: Neighborhood in Brugge. *Change it to Brugge?*

Berchem: District in Antwerp. *Should it be Antwerp (Berchem)?*

Erembodegem (Aalst): *Should it be Aalst (Erembodegem)?

Sint-Kruis (Brugge): *Should it be Brugge (Sint-Kruis)?

Westende (Middelkerke): *Should it be Middelkerke (Westende)?

Why is Gent (Ledeberg) not working ?


In [ ]:
orgs[orgs['Provincie van de organisatie_comment'] == 'Municipality Not Found'][['Gemeente van de organisatie']].iloc[:,0].unique()


## Adressen

Split Huisnummer and busnummer

In [ ]:
def split_house_bus_number(house_bus_number):
  house_number = bus_number = np.nan
  comment = []
  house_bus_number = house_bus_number.replace(' ', '')

  if ('z/n' not in house_bus_number and 'nan' not in house_bus_number) : 
    if (('bus' in house_bus_number or '/' in house_bus_number)) :
      comment.append('Splitting. Check it.')
      if ('bus' in house_bus_number) : 
        split = house_bus_number.split('bus')
      else :
        split = house_bus_number.split('/')
      house_number = split[0]
      bus_number = split[1]
    else:
        house_number = house_bus_number
    house_number =  house_number.replace('/', '').replace('-', '').replace(',', '')
      
  return [house_number, bus_number, ' - '.join(comment)]

In [ ]:
orgs[['Huisnr_cleansed', 'Busnr_new', 'Huisnr_comment']] = pd.DataFrame(orgs['Huisnr'].astype(str).apply(split_house_bus_number).values.tolist(), columns=['house_number', 'bus_number', 'comment'])

In [ ]:
orgs[orgs['Busnr_new'].notnull()][['Huisnr', 'Huisnr_cleansed', 'Busnr_new','Huisnr_comment']]

Postcode

In [ ]:
orgs['Postcode van de organisatie_cleansed'] = orgs['Postcode van de organisatie'].astype(str).str.replace('\.0', '').replace('nan', np.nan)

In [ ]:
orgs[['Postcode van de organisatie', 'Postcode van de organisatie_cleansed']]

NIScode

In [ ]:
orgs['NIScode_cleansed'] = orgs['NIScode'].astype(str).str.replace('\.0', '').replace('nan', np.nan)

KBOnr empty: **258**


In [ ]:
orgs[orgs['KBOnr_cleansed'].isnull()]

Unieke Naam empty: **166**

In [ ]:
orgs[orgs['Unieke Naam'].isnull()]

## Change Event

In [ ]:
def find_resulting_organisation(name, type_entiteit):
  if name == 'Puurs Sint-Amands':
    return orgs[(orgs['Unieke Naam'].str.contains('PUURS_SINT_AMANDS', flags=re.IGNORECASE, regex=True, na=False)) & (orgs['Organisatiestatus'] == 'Actief') & (orgs['Type Entiteit'] == type_entiteit)]
  elif type_entiteit == 'Gemeente':
    return orgs[(orgs['Unieke Naam'].str.contains('G_' + name, flags=re.IGNORECASE, regex=True, na=False)) & (orgs['Organisatiestatus'] == 'Actief')]
  else:
    return orgs[(orgs['Unieke Naam'].str.contains('O_' + name, flags=re.IGNORECASE, regex=True, na=False)) & (orgs['Organisatiestatus'] == 'Actief')]

In [ ]:
orgs['Resulting organisation'] = None

In [ ]:
for index, row in orgs[orgs['Organisatiestatus'] == 'gefusioneerd'].iterrows():
  if str(row['Opmerkingen ivm Organisatie']).startswith('Fusie'):
    resulting_city = row['Opmerkingen ivm Organisatie'].split('tot')[-1].strip()
    obj_resulting_org = find_resulting_organisation(resulting_city, row['Type Entiteit'])
    orgs.at[index, 'Resulting organisation'] = str(obj_resulting_org.iloc[0]['KBOnr_cleansed'])

In [ ]:
orgs['Resulting organisation'].unique()

## Status

In [ ]:
orgs['Organisatiestatus'].unique()

In [ ]:
orgs['Organisatiestatus'] = orgs['Organisatiestatus'].str.replace('gefusioneerd', 'Gefusioneerd')

## Finish

In [ ]:
orgs_cleased = orgs[~orgs['organisation_id'].isnull()]

In [ ]:
orgs_cleased.info()

# Helper functions

In [ ]:
import hashlib
import uuid

def concept_uri(base_uri, input):
  m = hashlib.md5()
  m.update(input.encode('utf-8'))

  return URIRef(base_uri + m.hexdigest())

def addLiteral(subject, predicate, object_value, datatype=None):
  if object_value != str(np.nan):
    if datatype == None:
      g.add((subject, predicate, Literal(object_value, lang='nl')))
    else:
      g.add((subject, predicate, Literal(object_value, datatype=datatype)))

In [ ]:
def existsContact(row):
  return ((str(row['Website Cleansed']) != str(np.nan)) or (str(row['Algemeen telefoonnr']) != str(np.nan)) or (str(row['Algemeen mailadres']) != str(np.nan)))

def existsAddress(row):
  return ((str(row['Straat']) != str(np.nan)) or (str(row['Huisnr_cleansed']) != str(np.nan)) or (str(row['Busnr_new']) != str(np.nan)) or
          (str(row['Postcode van de organisatie_cleansed']) != str(np.nan)) or (str(row['Gemeente van de organisatie']) != str(np.nan)) or
          (str(row['Provincie van de organisatie_cleansed']) != str(np.nan)))
  
def existsSite(row):
  return (existsAddress(row) or existsContact(row))

# Init graph

In [ ]:
#namspaces
org = Namespace('http://www.w3.org/ns/org#')
locn = Namespace('http://www.w3.org/ns/locn#')
dc_terms= Namespace('http://purl.org/dc/terms/')
schema = Namespace('http://schema.org/')
regorg = Namespace('http://www.w3.org/ns/regorg#')
person = Namespace('http://www.w3.org/ns/person#')
vcard = Namespace('http://www.w3.org/2006/vcard/ns#')
dbpedia = Namespace('https://dbpedia.org/ontology/')

organisatie = Namespace('https://data.vlaanderen.be/ns/organisatie#')
persoon = Namespace('https://data.vlaanderen.be/ns/persoon#')
adres = Namespace('https://data.vlaanderen.be/ns/adres#')
generiek = Namespace('https://data.vlaanderen.be/ns/generiek#')

#lblod = Namespace('https://contacthub-dev.lblod.info/id/')
lblod = Namespace('https://contacthub-qa.lblod.info/id/')
os = Namespace('https://data.vlaanderen.be/id/concept/organisatiestatus/')
oc = Namespace('https://data.vlaanderen.be/id/concept/OrganisatieClassificatie/')



In [ ]:
g = Graph()

In [ ]:
#for status in orgs_cleased['Organisatiestatus'].dropna().unique():
#    subject = concept_uri(os, status)
#    g.add((subject, RDF.type, SKOS.Concept))
#    g.add((subject, SKOS.prefLabel, Literal(status, lang='nl')))
 

In [ ]:
for status in orgs_cleased['Organisatiestatus'].dropna().unique():
  subject = concept_uri(os, status)
  g.add((subject, RDF.type, SKOS.Concept))
  g.add((subject, SKOS.prefLabel, Literal(status, lang='nl')))
  if status.startswith('Actief') or status.startswith('Valt Niet Meer'):
    g.add((subject, SKOS.broader, os.actief))
  else:
    g.add((subject, SKOS.broader, os.nietactief))

In [ ]:
for category in orgs_cleased['Type Entiteit'].dropna().unique():
  category = concept_uri(oc, category)
  g.add((category, RDF.type, SKOS.Concept))
  g.add((category, SKOS.prefLabel, Literal(category, lang='nl')))

# Mapping

In [ ]:
for index, row in orgs_cleased.iterrows():
  abb_id = concept_uri(lblod + 'organisatie/', str(row['organisation_id']))
  g.add((abb_id, RDF.type, org.Organization))

  if str(row['KBOnr_cleansed']) != str(np.nan):
    kbo_id = concept_uri(lblod + 'gestructureerdeIdentificator/', str(row['organisation_id']) + str(row['KBOnr_cleansed']))
    g.add((kbo_id, RDF.type, generiek.GestructureerdeIdentificator))
    addLiteral(kbo_id, generiek.lokaleIdentificator, str(row['KBOnr_cleansed']), XSD.string)

    g.add((abb_id, generiek.gestructureerdeIdentificator, kbo_id))

  g.add((abb_id, org.classification, concept_uri(oc, row['Type Entiteit'])))
  
  if str(row['Unieke Naam']) != str(np.nan):
    unieke_naam_id = concept_uri(lblod + 'gestructureerdeIdentificator/', str(row['organisation_id']) + str(row['Unieke Naam']) + '1')
    g.add((unieke_naam_id, RDF.type, generiek.GestructureerdeIdentificator))
    addLiteral(unieke_naam_id, generiek.lokaleIdentificator, str(row['Unieke Naam']), XSD.string)

    g.add((abb_id, generiek.gestructureerdeIdentificator, unieke_naam_id))

  if str(row['Unieke Naam van actieve organisaties']) != str(np.nan):
    unieke_naam_active_id = concept_uri(lblod + 'gestructureerdeIdentificator/', str(row['organisation_id']) + str(row['Unieke Naam van actieve organisaties']) + '2')
    g.add((unieke_naam_id, RDF.type, generiek.GestructureerdeIdentificator))
    addLiteral(unieke_naam_id, generiek.lokaleIdentificator, str(row['Unieke Naam van actieve organisaties']), XSD.string)

    g.add((abb_id, generiek.gestructureerdeIdentificator, unieke_naam_id))

  if existsSite(row):
    site_id = concept_uri(lblod + 'vesting/', str(row['organisation_id']))
    g.add((site_id, RDF.type, org.Site))

    if existsContact(row):
      contact_id = concept_uri(lblod + 'contactinfo/', str(row['organisation_id']))
      g.add((contact_id, RDF.type, schema.ContactPoint))
      
      addLiteral(contact_id, FOAF.page, str(row['Website Cleansed']), XSD.anyURI)
      addLiteral(contact_id, schema.telephone, str(row['Algemeen telefoonnr']), XSD.string)
      addLiteral(contact_id, schema.email, str(row['Algemeen mailadres']), XSD.string)

      g.add((site_id, schema.siteAddress, contact_id))

    if existsAddress(row):
      address_id = concept_uri(lblod + 'adresvoorstelling/', str(row['organisation_id']))
      g.add((address_id, RDF.type, locn.Address))
      addLiteral(address_id, locn.thoroughfare, str(row['Straat']))
      addLiteral(address_id, adres['Adresvoorstelling.huisnummer'], str(row['Huisnr_cleansed']), XSD.string)
      addLiteral(address_id, adres['Adresvoorstelling.busnummer'], str(row['Busnr_new']), XSD.string)
      addLiteral(address_id, locn.postCode, str(row['Postcode van de organisatie_cleansed']), XSD.string)
      addLiteral(address_id, adres.gemeentenaam, str(row['Gemeente van de organisatie']))
      addLiteral(address_id, locn.adminUnitL2, str(row['Provincie van de organisatie_cleansed']))
      g.add((address_id, adres.land, Literal('België', lang='nl')))

      g.add((site_id, organisatie.bestaatUit, address_id))
    
    g.add((abb_id, org.hasPrimarySite, site_id))

  if row['Unieke Naam'] != row['Moederentiteit']:
    find_moeder_kboid = orgs[orgs['Unieke Naam'] == row['Moederentiteit']]
    if len(find_moeder_kboid) > 0:
      g.add((abb_id, org.linkedTo, concept_uri(lblod + 'organisatie/', str(find_moeder_kboid.iloc[0]['organisation_id']))))

  g.add((abb_id, regorg.orgStatus, concept_uri(os, str(row['Organisatiestatus']))))

  addLiteral(abb_id, dbpedia.nisCode, str(row['NIScode_cleansed']), XSD.string)
  
  addLiteral(abb_id, regorg.legalName, str(row['Maatschappelijke Naam']))
  addLiteral(abb_id, SKOS.prefLabel, str(row['Titel']))

  if pd.notna(row['Actief vanaf']):
    change_event_open_id = concept_uri(lblod + 'veranderingsgebeurtenis/', str(row['organisation_id']) + str(row['Actief vanaf']))
    g.add((change_event_open_id, RDF.type, organisatie.Oprichting))
    addLiteral(change_event_open_id, DC.date, str(row['Actief vanaf']), XSD.date)
    g.add((abb_id, org.resultedFrom, change_event_open_id))

  if row['Organisatiestatus'] == 'Valt niet meer onder Vlaams toezicht':
    change_event_not_flemish_id = concept_uri(lblod + 'veranderingsgebeurtenis/', str(row['organisation_id']) + str(row['Actief tot']) + str(row['Actief tot']))
    g.add((change_event_not_flemish_id, RDF.type, organisatie.Vervanging))
    g.add((abb_id, org.resultedFrom, change_event_not_flemish_id))

  elif row['Organisatiestatus'] == 'Gefusioneerd':
    change_event_merged_id = concept_uri(lblod + 'veranderingsgebeurtenis/', str(row['organisation_id']) + str(row['Actief tot']))
    g.add((change_event_merged_id, RDF.type, organisatie.Fusie))
    if pd.notna(row['Actief tot']):
      addLiteral(change_event_merged_id, DC.date, str(row['Actief tot']), XSD.date)
    # addLiteral(change_event_merged_id, NEED PROPERTY, 'Opmerkingen ivm Organisatie')
  
    merged_abb_id = concept_uri(lblod + 'organisatie/', str(row['Resulting organisation']))
    g.add((change_event_merged_id, org.originalOrganization, abb_id))
    g.add((change_event_merged_id, org.resultingOrganization, merged_abb_id))
    g.add((abb_id, org.changedBy, change_event_merged_id))
  elif pd.notna(row['Actief tot']):
    change_event_close_id = concept_uri(lblod + 'veranderingsgebeurtenis/', str(row['organisation_id']) + str(row['Actief tot']))
    g.add((change_event_close_id, RDF.type, organisatie.Stopzetting))
    addLiteral(change_event_close_id, DC.date, str(row['Actief tot']), XSD.date)
    g.add((abb_id, org.changedBy, change_event_close_id))


In [ ]:
g.serialize('organisatie-qa.ttl',format='turtle')